In [7]:
import warnings

warnings.filterwarnings('ignore')

In [1]:
from sqltoolkit.connectors import AzureSQLConnector
from sqltoolkit.dbutils import DatabaseUtils
  
# Azure SQL Connection  
server = 'vince-h-sql.database.windows.net'  
database = 'adventureworks'  
azure_connector = AzureSQLConnector(server=server, database=database)  
db_utils = DatabaseUtils(azure_connector)  
  
# List tables in the database  
print(db_utils.list_database_tables())

[{"TABLE_NAME": "Customer"}, {"TABLE_NAME": "ProductModel"}, {"TABLE_NAME": "ProductDescription"}, {"TABLE_NAME": "Product"}, {"TABLE_NAME": "ProductModelProductDescription"}, {"TABLE_NAME": "ProductCategory"}, {"TABLE_NAME": "BuildVersion"}, {"TABLE_NAME": "ErrorLog"}, {"TABLE_NAME": "Address"}, {"TABLE_NAME": "CustomerAddress"}, {"TABLE_NAME": "SalesOrderDetail"}, {"TABLE_NAME": "SalesOrderHeader"}]


/home/vhoudebine/azure_openai/azure-nl2sql-accelerator/sqltoolkit/dbutils.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.connection)
